In [ ]:
################ Directly Save the file in Google Drive, need not to download it again to use

################################ Use this for encryption #############################


##################################
# Import necessary libraries
import csv
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the filename and the specific path on Google Drive
filename = 'Dataset_salsa_100000_3round.csv'
google_drive_path = '/content/drive/My Drive/datafile/'  # Change 'specific_folder' to your desired folder path

# Ensure the folder exists (create if it doesn't)
import os
if not os.path.exists(google_drive_path):
    os.makedirs(google_drive_path)

# Full path for saving the file
full_path = os.path.join(google_drive_path, filename)




#############################################
















#Basic functions
import random
import ctypes

# Define constants
ul = ctypes.c_uint

MOD = 4294967296


# Define rotateleft and rotateright functions
def rotateleft(x, n):
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF

def rotateright(x, n):
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

# # Define update function
# def update(a, b, c, n):
#     return (a ^ rotateleft((b + c), n))


# Define update function
def update(a, b, c, n):
    return (a ^ rotateleft((b + c) & 0xFFFFFFFF, n)) & 0xFFFFFFFF


def copystate(x1, x):
    for i in range(16):
        x1[i] = x[i]

# Define myrand32 as a lambda function
myrand32 = lambda: ul(int(4294967296.0 * random.random()))

# Define initialization function
def initializeR(x):


    for i in range(16):
        x[i] = myrand32()

    x[0] = 0x61707865
    x[5] = 0x3320646e
    x[10] = 0x79622d32
    x[15] = 0x6b206574



def RANDOM(x):


    for i in range(16):
        x[i] = myrand32()



# Define qr and inqr functions
def qr(x0, x1, x2, x3):
    z1 = update(x1, x3, x0, 7)
    z2 = update(x2, x0, z1, 9)
    z3 = update(x3, z1, z2, 13)
    z0 = update(x0, z2, z3, 18)

    return z0, z1, z2, z3

def inqr(x0, x1, x2, x3):
    z0 = update(x0, x3, x2, 18)
    z3 = update(x3, x2, x1, 13)
    z2 = update(x2, x1, z0, 9)
    z1 = update(x1, z0, z3, 7)

    return z0, z1, z2, z3

# Define columnround and incolumnround functions
def columnround(x):
    x[0], x[4], x[8], x[12] = qr(x[0], x[4], x[8], x[12])
    x[5], x[9], x[13], x[1] = qr(x[5], x[9], x[13], x[1])
    x[10], x[14], x[2], x[6] = qr(x[10], x[14], x[2], x[6])
    x[15], x[3], x[7], x[11] = qr(x[15], x[3], x[7], x[11])

def incolumnround(x):
    x[0], x[4], x[8], x[12] = inqr(x[0], x[4], x[8], x[12])
    x[5], x[9], x[13], x[1] = inqr(x[5], x[9], x[13], x[1])
    x[10], x[14], x[2], x[6] = inqr(x[10], x[14], x[2], x[6])
    x[15], x[3], x[7], x[11] = inqr(x[15], x[3], x[7], x[11])

# Define rowround and inrowround functions
def rowround(x):
    x[0], x[1], x[2], x[3] = qr(x[0], x[1], x[2], x[3])
    x[5], x[6], x[7], x[4] = qr(x[5], x[6], x[7], x[4])
    x[10], x[11], x[8], x[9] = qr(x[10], x[11], x[8], x[9])
    x[15], x[12], x[13], x[14] = qr(x[15], x[12], x[13], x[14])

def inrowround(x):
    x[0], x[1], x[2], x[3] = inqr(x[0], x[1], x[2], x[3])
    x[5], x[6], x[7], x[4] = inqr(x[5], x[6], x[7], x[4])
    x[10], x[11], x[8], x[9] = inqr(x[10], x[11], x[8], x[9])
    x[15], x[12], x[13], x[14] = inqr(x[15], x[12], x[13], x[14])


# # Example usage
# if __name__ == "__main__":
#     x = (ul * 16)()  # array of 16 unsigned ints
#     initializeR(x)
#     print('Initial state:')
#     for i in range(16):
#         print(hex(x[i]))
#     columnround(x)
#     print('After columnround:')
#     for i in range(16):
#         print(hex(x[i]))
#     incolumnround(x)
#     print('After incolumnround:')
#     for i in range(16):
#         print(hex(x[i]))
#     rowround(x)
#     print('After rowround:')
#     for i in range(16):
#         print(hex(x[i]))
#     inrowround(x)
#     print('After inrowround:')
#     for i in range(16):
#         print(hex(x[i]))




##################################

def encrypt(x,r):
    #initializeR(x)
    if r%2 == 0:
       for i in range(r//2):
           columnround(x)
           rowround(x)

    else:
      for i in range((r-1)//2):
          columnround(x)
          rowround(x)
      columnround(x)
    return x


############################################


import math, os,sys
import random


C = (ul*16)()

import numpy as np


############## Real Ciphertext-Pair Generation ##############

def real_plain(P1, r):
  l1 = []
  delta = (ul*16)()
  #delta = [0]*16

  X1 = (ul*16)()
  X2 = (ul*16)()
  copystate(X1,P1)

  P2 = (ul*16)()
  copystate(P2,P1)
  P2[7] = P2[7] ^ (0x00000001 << 31)

  copystate(X2,P2)

  C1 = (ul*16)()
  C2 = (ul*16)()
  C1 = encrypt(P1,r)
  C2 = encrypt(P2,r)


  # for i in range(16):
  #   C1[i] = C1[i] ^ X1[i]
  #   C2[i] = C2[i] ^ X2[i]





  Label = 1

  for i in range(16):
    delta[i] = C1[i] ^ C2[i]

  #Integer conversion for Delta

  str_delta_0 = str(delta[0])
  str_delta_0 = int(str_delta_0)


  str_delta_1 = str(delta[1])
  str_delta_1 = int(str_delta_1)


  str_delta_2 = str(delta[2])
  str_delta_2 = int(str_delta_2)

  str_delta_3 = str(delta[3])
  str_delta_3 = int(str_delta_3)


  str_delta_4 = str(delta[4])
  str_delta_4 = int(str_delta_4)


  str_delta_5 = str(delta[5])
  str_delta_5 = int(str_delta_5)


  str_delta_6 = str(delta[6])
  str_delta_6 = int(str_delta_6)

  str_delta_7 = str(delta[7])
  str_delta_7 = int(str_delta_7)


  str_delta_8 = str(delta[8])
  str_delta_8 = int(str_delta_8)


  str_delta_9 = str(delta[9])
  str_delta_9 = int(str_delta_9)


  str_delta_10 = str(delta[10])
  str_delta_10 = int(str_delta_10)

  str_delta_11 = str(delta[11])
  str_delta_11 = int(str_delta_11)


  str_delta_12 = str(delta[12])
  str_delta_12 = int(str_delta_12)


  str_delta_13 = str(delta[13])
  str_delta_13 = int(str_delta_13)


  str_delta_14 = str(delta[14])
  str_delta_14 = int(str_delta_14)

  str_delta_15 = str(delta[15])
  str_delta_15 = int(str_delta_15)


  #Integer conversion for Cipher 1



  str_C1_0 = str(C1[0])
  str_C1_0 = int(str_C1_0)

  str_C1_1 = str(C1[1])
  str_C1_1 = int(str_C1_1)

  str_C1_2 = str(C1[2])
  str_C1_2 = int(str_C1_2)

  str_C1_3 = str(C1[3])
  str_C1_3 = int(str_C1_3)

  str_C1_4 = str(C1[4])
  str_C1_4 = int(str_C1_4)

  str_C1_5 = str(C1[5])
  str_C1_5 = int(str_C1_5)

  str_C1_6 = str(C1[6])
  str_C1_6 = int(str_C1_6)

  str_C1_7 = str(C1[7])
  str_C1_7 = int(str_C1_7)

  str_C1_8 = str(C1[8])
  str_C1_8 = int(str_C1_8)

  str_C1_9 = str(C1[9])
  str_C1_9 = int(str_C1_9)

  str_C1_10 = str(C1[10])
  str_C1_10 = int(str_C1_10)

  str_C1_11 = str(C1[11])
  str_C1_11 = int(str_C1_11)

  str_C1_12 = str(C1[12])
  str_C1_12 = int(str_C1_12)

  str_C1_13 = str(C1[13])
  str_C1_13 = int(str_C1_13)

  str_C1_14 = str(C1[14])
  str_C1_14 = int(str_C1_14)

  str_C1_15 = str(C1[15])
  str_C1_15 = int(str_C1_15)



  #Integer conversion for Cipher 2



  str_C2_0 = str(C2[0])
  str_C2_0 = int(str_C2_0)

  str_C2_1 = str(C2[1])
  str_C2_1 = int(str_C2_1)

  str_C2_2 = str(C2[2])
  str_C2_2 = int(str_C2_2)

  str_C2_3 = str(C2[3])
  str_C2_3 = int(str_C2_3)

  str_C2_4 = str(C2[4])
  str_C2_4 = int(str_C2_4)

  str_C2_5 = str(C2[5])
  str_C2_5 = int(str_C2_5)

  str_C2_6 = str(C2[6])
  str_C2_6 = int(str_C2_6)

  str_C2_7 = str(C2[7])
  str_C2_7 = int(str_C2_7)

  str_C2_8 = str(C2[8])
  str_C2_8 = int(str_C2_8)

  str_C2_9 = str(C2[9])
  str_C2_9 = int(str_C2_9)

  str_C2_10 = str(C2[10])
  str_C2_10 = int(str_C2_10)

  str_C2_11 = str(C2[11])
  str_C2_11 = int(str_C2_11)

  str_C2_12 = str(C2[12])
  str_C2_12 = int(str_C2_12)

  str_C2_13 = str(C2[13])
  str_C2_13 = int(str_C2_13)

  str_C2_14 = str(C2[14])
  str_C2_14 = int(str_C2_14)

  str_C2_15 = str(C2[15])
  str_C2_15 = int(str_C2_15)





  l1.append(str_delta_0)
  l1.append(str_delta_1)
  l1.append(str_delta_2)
  l1.append(str_delta_3)
  l1.append(str_delta_4)
  l1.append(str_delta_5)
  l1.append(str_delta_6)
  l1.append(str_delta_7)
  l1.append(str_delta_8)
  l1.append(str_delta_9)
  l1.append(str_delta_10)
  l1.append(str_delta_11)
  l1.append(str_delta_12)
  l1.append(str_delta_13)
  l1.append(str_delta_14)
  l1.append(str_delta_15)



  l1.append(str_C1_0)
  l1.append(str_C1_1)
  l1.append(str_C1_2)
  l1.append(str_C1_3)
  l1.append(str_C1_4)
  l1.append(str_C1_5)
  l1.append(str_C1_6)
  l1.append(str_C1_7)
  l1.append(str_C1_8)
  l1.append(str_C1_9)
  l1.append(str_C1_10)
  l1.append(str_C1_11)
  l1.append(str_C1_12)
  l1.append(str_C1_13)
  l1.append(str_C1_14)
  l1.append(str_C1_15)


  l1.append(str_C2_0)
  l1.append(str_C2_1)
  l1.append(str_C2_2)
  l1.append(str_C2_3)
  l1.append(str_C2_4)
  l1.append(str_C2_5)
  l1.append(str_C2_6)
  l1.append(str_C2_7)
  l1.append(str_C2_8)
  l1.append(str_C2_9)
  l1.append(str_C2_10)
  l1.append(str_C2_11)
  l1.append(str_C2_12)
  l1.append(str_C2_13)
  l1.append(str_C2_14)
  l1.append(str_C2_15)


  l1.append(Label)
  listoflists.append(l1)

########### Random Ciphertext-Pair Generation #######################


def random_plain(P1, r):
  l1 = []
  delta = (ul*16)()
  #delta = [0]*16

  P2 = (ul*16)()


  initializeR(P2)
  #RANDOM(P1)
  #RANDOM(P2)


  X1 = (ul*16)()
  X2 = (ul*16)()
  copystate(X1,P1)
  copystate(X2,P2)

  C1 = (ul*16)()
  C2 = (ul*16)()
  C1 = encrypt(P1,r)
  C2 = encrypt(P2,r)
  #C1 = P1
  #C2 = P2




  # for i in range(16):
  #   C1[i] = C1[i] ^ X1[i]
  #   C2[i] = C2[i] ^ X2[i]

  Label = 0

  for i in range(16):
    delta[i] = C1[i] ^ C2[i]

  #Integer conversion for Delta

  str_delta_0 = str(delta[0])
  str_delta_0 = int(str_delta_0)


  str_delta_1 = str(delta[1])
  str_delta_1 = int(str_delta_1)


  str_delta_2 = str(delta[2])
  str_delta_2 = int(str_delta_2)

  str_delta_3 = str(delta[3])
  str_delta_3 = int(str_delta_3)


  str_delta_4 = str(delta[4])
  str_delta_4 = int(str_delta_4)


  str_delta_5 = str(delta[5])
  str_delta_5 = int(str_delta_5)


  str_delta_6 = str(delta[6])
  str_delta_6 = int(str_delta_6)

  str_delta_7 = str(delta[7])
  str_delta_7 = int(str_delta_7)


  str_delta_8 = str(delta[8])
  str_delta_8 = int(str_delta_8)


  str_delta_9 = str(delta[9])
  str_delta_9 = int(str_delta_9)


  str_delta_10 = str(delta[10])
  str_delta_10 = int(str_delta_10)

  str_delta_11 = str(delta[11])
  str_delta_11 = int(str_delta_11)


  str_delta_12 = str(delta[12])
  str_delta_12 = int(str_delta_12)


  str_delta_13 = str(delta[13])
  str_delta_13 = int(str_delta_13)


  str_delta_14 = str(delta[14])
  str_delta_14 = int(str_delta_14)

  str_delta_15 = str(delta[15])
  str_delta_15 = int(str_delta_15)


  #Integer conversion for Cipher 1



  str_C1_0 = str(C1[0])
  str_C1_0 = int(str_C1_0)

  str_C1_1 = str(C1[1])
  str_C1_1 = int(str_C1_1)

  str_C1_2 = str(C1[2])
  str_C1_2 = int(str_C1_2)

  str_C1_3 = str(C1[3])
  str_C1_3 = int(str_C1_3)

  str_C1_4 = str(C1[4])
  str_C1_4 = int(str_C1_4)

  str_C1_5 = str(C1[5])
  str_C1_5 = int(str_C1_5)

  str_C1_6 = str(C1[6])
  str_C1_6 = int(str_C1_6)

  str_C1_7 = str(C1[7])
  str_C1_7 = int(str_C1_7)

  str_C1_8 = str(C1[8])
  str_C1_8 = int(str_C1_8)

  str_C1_9 = str(C1[9])
  str_C1_9 = int(str_C1_9)

  str_C1_10 = str(C1[10])
  str_C1_10 = int(str_C1_10)

  str_C1_11 = str(C1[11])
  str_C1_11 = int(str_C1_11)

  str_C1_12 = str(C1[12])
  str_C1_12 = int(str_C1_12)

  str_C1_13 = str(C1[13])
  str_C1_13 = int(str_C1_13)

  str_C1_14 = str(C1[14])
  str_C1_14 = int(str_C1_14)

  str_C1_15 = str(C1[15])
  str_C1_15 = int(str_C1_15)



  #Integer conversion for Cipher 2



  str_C2_0 = str(C2[0])
  str_C2_0 = int(str_C2_0)

  str_C2_1 = str(C2[1])
  str_C2_1 = int(str_C2_1)

  str_C2_2 = str(C2[2])
  str_C2_2 = int(str_C2_2)

  str_C2_3 = str(C2[3])
  str_C2_3 = int(str_C2_3)

  str_C2_4 = str(C2[4])
  str_C2_4 = int(str_C2_4)

  str_C2_5 = str(C2[5])
  str_C2_5 = int(str_C2_5)

  str_C2_6 = str(C2[6])
  str_C2_6 = int(str_C2_6)

  str_C2_7 = str(C2[7])
  str_C2_7 = int(str_C2_7)

  str_C2_8 = str(C2[8])
  str_C2_8 = int(str_C2_8)

  str_C2_9 = str(C2[9])
  str_C2_9 = int(str_C2_9)

  str_C2_10 = str(C2[10])
  str_C2_10 = int(str_C2_10)

  str_C2_11 = str(C2[11])
  str_C2_11 = int(str_C2_11)

  str_C2_12 = str(C2[12])
  str_C2_12 = int(str_C2_12)

  str_C2_13 = str(C2[13])
  str_C2_13 = int(str_C2_13)

  str_C2_14 = str(C2[14])
  str_C2_14 = int(str_C2_14)

  str_C2_15 = str(C2[15])
  str_C2_15 = int(str_C2_15)





  l1.append(str_delta_0)
  l1.append(str_delta_1)
  l1.append(str_delta_2)
  l1.append(str_delta_3)
  l1.append(str_delta_4)
  l1.append(str_delta_5)
  l1.append(str_delta_6)
  l1.append(str_delta_7)
  l1.append(str_delta_8)
  l1.append(str_delta_9)
  l1.append(str_delta_10)
  l1.append(str_delta_11)
  l1.append(str_delta_12)
  l1.append(str_delta_13)
  l1.append(str_delta_14)
  l1.append(str_delta_15)



  l1.append(str_C1_0)
  l1.append(str_C1_1)
  l1.append(str_C1_2)
  l1.append(str_C1_3)
  l1.append(str_C1_4)
  l1.append(str_C1_5)
  l1.append(str_C1_6)
  l1.append(str_C1_7)
  l1.append(str_C1_8)
  l1.append(str_C1_9)
  l1.append(str_C1_10)
  l1.append(str_C1_11)
  l1.append(str_C1_12)
  l1.append(str_C1_13)
  l1.append(str_C1_14)
  l1.append(str_C1_15)


  l1.append(str_C2_0)
  l1.append(str_C2_1)
  l1.append(str_C2_2)
  l1.append(str_C2_3)
  l1.append(str_C2_4)
  l1.append(str_C2_5)
  l1.append(str_C2_6)
  l1.append(str_C2_7)
  l1.append(str_C2_8)
  l1.append(str_C2_9)
  l1.append(str_C2_10)
  l1.append(str_C2_11)
  l1.append(str_C2_12)
  l1.append(str_C2_13)
  l1.append(str_C2_14)
  l1.append(str_C2_15)


  l1.append(Label)
  listoflists.append(l1)



import sys
listoflists = []
iterations = 100000
r = 3
for i in range(iterations):
  P1 = (ul*16)()
  initializeR(P1)
  y = random.randint(0,1)
  if y == 0:
    random_plain(P1, r)
  else:
    real_plain(P1, r)


header = ['del_0','del_1','del_2','del_3','del_4','del_5','del_6','del_7','del_8','del_9','del_10','del_11','del_12','del_13','del_14','del_15',
          'C1_0','C1_1','C1_2','C1_3','C1_4','C1_5','C1_6','C1_7','C1_8','C1_9','C1_10','C1_11','C1_12','C1_13','C1_14','C1_15',
          'C2_0','C2_1','C2_2','C2_3','C2_4','C2_5','C2_6','C2_7','C2_8','C2_9','C2_10','C2_11','C2_12','C2_13','C2_14','C2_15','Label']

# import csv
# #print(listoflists)

# filename = 'Dataset_salsa_1000000_6round.csv'
# with open(filename, 'w', newline="") as file:
#     csvwriter = csv.writer(file) #  create a csvwriter object
#     csvwriter.writerow(header) #  write the header
#     csvwriter.writerows(listoflists) #  write the rest of the data


with open(full_path, 'w', newline="") as file:
    csvwriter = csv.writer(file)  # create a csvwriter object
    csvwriter.writerow(header)  # write the header
    csvwriter.writerows(listoflists)  # write the rest of the data

print(f'File saved to {full_path}')
